In [1]:
import wandb
#from wandb import Api
from collections import defaultdict
import pandas as pd
import numpy as np

import random

import seaborn as sns
import matplotlib.pyplot as plt
from IPython.display import HTML
from IPython.display import display_html
import os

import warnings
warnings.filterwarnings('ignore')

%load_ext autoreload
%autoreload 2

In [2]:
HUES = {}
# HUES["GCN-Transformer"] = "Red"
# HUES["GCN-Avg"] = "Purple"
# HUES["GCN"] = "Black"
# #HUES["DBMS (instance-tuned)"] = "Gray"

# HUES["DBMS-all"] = "Gray"


# HUES["GCN-Pretrained-Latent"] = "Blue"
# HUES["GCN-Pretrained"] = "Green"
# HUES["GCN-PretrainedT-CurrentDB"] = "Gray"
# HUES["GCN-PretrainedT-NoCurrentDB"] = "Green"
# HUES["GCN-PreatinedT-NoCurrentDB-NoActualFeats"] = "Black"


BASELINES = ["DBMS (TestEnv)", "DBMS (TestDB + TestEnv)"]

HUES["DBMS (TestEnv)"] = "Red"
HUES["DBMS (TestDB + TestEnv)"] = "Violet"

#HUES["Pretrained Environment Transformer"] = "Green"

HUES["GCN (TestDB)"] = "Black"
HUES["GCN + LPT"] = "Green"
HUES["GCN (TestDB + TestEnv)"] = "Purple"
HUES["GCN-Transformer (TestDB)"] = "Gray"

In [3]:
api = wandb.Api()

In [4]:
bruns = api.runs("pari/LatencyPredictor",
    {"$and": [
      {"config.alg": "dbms"},
      {"$or": [
            {"$and": [
                {"tags":"single-final9-min-01-baseline"},
                #{"tags":"single-stack-final9-min-01-baseline"},
            ]},
      ]} 
    ]
    })

print(f"Found {len(bruns)} runs")

Found 3 runs


In [5]:
import time
start = time.time()

bdfs = []

cur_run = None
for run in bruns:
    skip = False
    if skip:
        continue
    if run.State != "finished":
        print("not finished!")
        cur_run = run
        continue
    
    data = defaultdict(list)
    data["Tags"].append(run.Tags)
    data["name"].append(run.name)
    
    
    for k,v in run.config.items():
        if not k.startswith("_") or not k.contains("/"):
            data[k].append(v)

    for k,v in run.summary._json_dict.items():
        if k.startswith("_"):
            continue
        if "/" in k:
            continue
        data[k].append(v)
        
    bdfs.append(pd.DataFrame(data))

print("took: ", time.time()-start)
print(len(bdfs))

bdf = pd.concat(bdfs)
#bdf = bdf[bdf["config"] == "configs/baselines/config_dbms_imdb.yaml"]
print(len(bdf))

took:  0.02343010902404785
3
3


In [6]:
bdf[["Final-LatencyQError-test-mean", "Final-LatencyQError-test_unseen-mean", "dbms_fit_test"]]

,Final-LatencyQError-test-mean,Final-LatencyQError-test_unseen-mean,dbms_fit_test
0,18.042923,26.897645,0
0,18.042923,NaN,0
0,20.202283,17.142025,1


In [7]:
runs = api.runs("pari/LatencyPredictor",
    {"$and": [
      #{"config.eval_epoch": 5},
      #{"config.traindata_dir": "LatencyCollectorResults/background"},
      #{"config.traindata_dir": "LatencyCollectorResults/new"},
      {"$or": [
            {"$or": [
                {"tags":"imdb-ablation-data2"},
                #{"tags":"imdb-final9-ablations"},
            ]},
      ]} 
    ]
    })

print(f"Found {len(runs)} runs")

Found 12 runs


In [8]:
import time
start = time.time()

dfs = []

cur_run = None
for run in runs:
    skip = False
    if skip:
        continue
    if run.State != "finished":
        cur_run = run
        continue
    
    data = defaultdict(list)
    data["Tags"].append(run.Tags)
    data["name"].append(run.name)
    
    
    for k,v in run.config.items():
        if not k.startswith("_") or not k.contains("/"):
            data[k].append(v)

    for k,v in run.summary._json_dict.items():
        if k.startswith("_"):
            continue
        if "/" in k:
            continue
        data[k].append(v)
        
    dfs.append(pd.DataFrame(data))

print("took: ", time.time()-start)
print(len(dfs))

took:  0.058259010314941406
12


In [9]:
df = pd.concat(dfs)

df1 = df[df["config"].str.contains("transfer")]
df1 = df[df["feat_onehot"] == 0]
print("Num transferrable: ", len(df1))

df2 = df[(df["tags"] == "exp1_single,exp2_single,exp3_single,exp7_single_ceb2") &
            (df["split_kind"] == "lt_type-query") &
            ~(df["config"].str.contains("nopretrain"))
        ]

df3 = df[df["config"].str.contains("nopretrain")]
df3 = df3[df3["sys_net_pretrained"] == 0]

# df4 = df[(df["split_kind"] == "lt_type-query") & \
#           ~(df["config"].str.contains("nopretrain"))
#         ]

print(set(df3["seed"]))
df3 = df3.drop_duplicates(["seed"])
print(len(df3))

df = pd.concat([df1,df2, df3])

df = pd.concat([df, bdf])
print(len(df))

Num transferrable:  0
set()
0
15


In [11]:
set(df["alg"])

{'dbms', 'nn'}

In [12]:
# GCN, GCN-Transformer, GCN-Pretrained, GCN-Pretrained-Latent

PRETRAINED_NAME = False

def get_alg(row):
    if "dbms" == row["alg"] or "dbms-c" == row["alg"]:
        if row["dbms_fit_test"]:
            return "DBMS\n(Test DB,Test Env)"
        else:
            return "DBMS\n(Diff DB,Test Env)"
    
    elif "dbms-all" == row["alg"]:
        return "DBMS-all"
    
    elif "avg" in row["config"]:
        return "GCN-Avg"
    
    elif row["arch"] == "gcn":
        if "train_local_model" in row:
            if "train_env_model" in row and str(row["train_env_model"]) != "nan" and row["train_env_model"]:
                print(row["train_env_model"])
                return "GCN\n(Diff DB,Test Env)"
            elif str(row["train_local_model"]) == "nan" or not row["train_local_model"]:
                return "GCN\n(Test DB,Diff Env)"
            else:
                return "GCN\n(Test DB,Test Env)"
        else:
            return "GCN"
    
    elif row["sys_net_pretrained"] == 0:
        if row["extra_training"]:
            return "Factorized\n(No Pre-training)"
        else:
            return "GCN-Transformer (TestDB)"
    
    elif "evals" in row["config"]:
        if PRETRAINED_NAME:
            pn = row["sys_net_pretrained_fn"]
            mn = "GCN-Pretrained-" + str(row["lr"]) + "extra-training-" + str(row["extra_training"])
            mn += "wd-" + str(row["weight_decay"])
            mn += "-" + pn
            return mn
        else:
            if row["sys_net_pretrained_fn"] == "models4/concurrent_noimdb_randomsplit_test90.wt":
                return "Factorized (LPT)"
            elif row["sys_net_pretrained_fn"] == "models5/mixed_nostack_logfeats_cols.wt":
                return "Factorized (LPT)"
            else:
                return row["sys_net_pretrained_fn"]
    else:
        return None
    
def get_final_alg(row):
    if "dbms" == row["alg"] or "dbms-c" == row["alg"]:
        if row["dbms_fit_test"]:
            return "DBMS\n(Test DB,Test Env)"
        else:
            return "DBMS\n(Diff DB,Test Env)"
    
    elif "dbms-all" == row["alg"]:
        return "DBMS-all"
    
    elif "avg" in row["config"]:
        return "GCN-Avg"
    
    elif row["arch"] == "gcn":
        if "train_local_model" in row:
            if "train_env_model" in row and str(row["train_env_model"]) != "nan" and row["train_env_model"]:
                print(row["train_env_model"])
                return "GCN\n(Diff DB,Test Env)"
            elif str(row["train_local_model"]) == "nan" or not row["train_local_model"]:
                return "GCN\n(Test DB,Diff Env)"
            else:
                return "GCN\n(Test DB,Test Env)"
        else:
            return "GCN"
    
    elif row["sys_net_pretrained"] == 0:
        if row["extra_training"]:
            return "Factorized\n(No Pre-training)"
        else:
            return "GCN-Transformer (TestDB)"
    
    elif "evals" in row["config"]:
        if PRETRAINED_NAME:
            pn = row["sys_net_pretrained_fn"]
            mn = "GCN-Pretrained-" + str(row["lr"]) + "extra-training-" + str(row["extra_training"])
            mn += "wd-" + str(row["weight_decay"])
            mn += "-" + pn
            return mn
        else:
            if row["sys_net_pretrained_fn"] == "models4/concurrent_noimdb_randomsplit_test90.wt":
                return "Factorized (LPT)"
            elif row["sys_net_pretrained_fn"] == "models5/mixed_nostack_logfeats_cols.wt":
                return "Factorized (LPT)"
            else:
                return row["sys_net_pretrained_fn"]
    else:
        return None

def get_workload(row):
    if "dbms" in row["alg"].lower():
        return "IMDb"
    
    if "stack" in row["tags"]:
        return "Stack"
    elif "tpch" in row["tags"]:
        return "Tpch"
    elif "zdb" in row["tags"]:
        return "zdbs"
    else:
        return "IMDb"
    
    
df["Estimator"] = df.apply(lambda x: get_final_alg(x),axis=1)
df["alg"] = df.apply(lambda x: get_alg(x),axis=1)
df["workload"] = df.apply(lambda x: get_workload(x),axis=1)

In [13]:
df = df[df["alg"].notna()]

In [14]:
print(len(df))
print(set(df["Estimator"]))
print(set(df["num_instances"]))
print(set(df["seed"]))
print(set(df["config"]))
print("pretrain: " , set(df["sys_net_pretrained_fn"]))
print(set(df["alg"]))
print(set(df["tags"]))
print(set(df["workload"]))
print(set(df["tags"]))
print(set(df["seed"]))
print(df.keys())

15
{'models4/ablation_skipwk_tpch.wt', 'models4/ablation_skipwk_stack.wt', 'DBMS\n(Diff DB,Test Env)', 'DBMS\n(Test DB,Test Env)', 'models4/ablation_skipwk_ccs,accidents,airline,ssb,credit,baseball,basketball,movielens,seznam,financial.wt'}
{1}
{1, 2, 3, 4}
{'configs/baselines/config_dbms_imdb.yaml', 'configs/evals/single/config_noimdb_cols.yaml'}
pretrain:  {'models4/ablation_skipwk_tpch.wt', 'models4/ablation_skipwk_stack.wt', 'models4/noactual_mixed_noimdb_logfeats_col.wt', 'models4/ablation_skipwk_ccs,accidents,airline,ssb,credit,baseball,basketball,movielens,seznam,financial.wt', None}
{'models4/ablation_skipwk_tpch.wt', 'models4/ablation_skipwk_stack.wt', 'DBMS\n(Diff DB,Test Env)', 'DBMS\n(Test DB,Test Env)', 'models4/ablation_skipwk_ccs,accidents,airline,ssb,credit,baseball,basketball,movielens,seznam,financial.wt'}
{'exp1_single,exp2_single,exp3_single,exp7_single_ceb2', 'exp5_single_stats,exp8_single_stack,exp9_single_tpch,exp4_single_ergast,exp1_single,exp2_single,exp3_singl

In [15]:
#df = df[df["alg"] != "DBMS-all"]
#df = df[df["alg"] != "GCN-Pretrained-Latent"]

In [16]:
print(set(df["sys_net_pretrained_fn"]))
#print(set(df["Tag"]))

{'models4/ablation_skipwk_tpch.wt', 'models4/ablation_skipwk_stack.wt', 'models4/noactual_mixed_noimdb_logfeats_col.wt', 'models4/ablation_skipwk_ccs,accidents,airline,ssb,credit,baseball,basketball,movielens,seznam,financial.wt', None}


In [17]:
#df = df[df["weight_decay"] == 0.1]

In [18]:
pd.set_option('display.max_rows', 500)

tmp = df[df["num_instances"] == 1]
tmp[["seed", "num_instances", "alg", "extra_training", "weight_decay", "lr", "Final-LatencyQError-test-mean", "Final-LatencyAE-test-mean" , "weight_decay", "Final-LatencyMSE-test-mean", 
    "Final-LatencyQError-test-median", "Final-LatencyQError-test-99p"]].sort_values(by="seed")

,seed,num_instances,alg,extra_training,weight_decay,lr,Final-LatencyQError-test-mean,Final-LatencyAE-test-mean,weight_decay,Final-LatencyMSE-test-mean,Final-LatencyQError-test-median,Final-LatencyQError-test-99p
0,1,1,models4/ablation_skipwk_tpch.wt,0,1,0.0001,6.054534,52.728495,1,13403.299857,2.736873,51.907733
0,1,1,models4/ablation_skipwk_stack.wt,0,1,0.0001,8.088043,79.782357,1,26382.951061,3.063748,72.038626
0,1,1,"models4/ablation_skipwk_ccs,accidents,airline,...",0,1,0.0001,13.964419,107.968258,1,39820.831845,3.963645,183.318329
0,1,1,"DBMS\n(Diff DB,Test Env)",None,1,0.0001,18.042923,52.384090,1,10841.419676,4.467091,337.890212
0,1,1,"DBMS\n(Diff DB,Test Env)",None,1,0.0001,18.042923,52.384090,1,10841.419676,4.467091,337.890212
0,1,1,"DBMS\n(Test DB,Test Env)",0,1,0.0001,20.202283,59.208805,1,10288.468838,3.736298,334.147077
0,2,1,models4/ablation_skipwk_tpch.wt,0,1,0.0001,11.867752,52.275170,1,13114.341467,3.772218,153.663578
0,2,1,models4/ablation_skipwk_stack.wt,0,1,0.0001,6.697228,47.058851,1,11669.722128,2.451810,76.862999
0,2,1,"models4/ablation_skipwk_ccs,accidents,airline,...",0,1,0.0001,10.369030,51.588436,1,12895.952554,3.521946,101.445098
0,3,1,models4/ablation_skipwk_tpch.wt,0,1,0.0001,7.655630,50.221579,1,12658.340348,2.780142,78.374692


In [19]:
print(set(df["alg"]))
print(set(df["Estimator"]))

{'models4/ablation_skipwk_tpch.wt', 'models4/ablation_skipwk_stack.wt', 'DBMS\n(Diff DB,Test Env)', 'DBMS\n(Test DB,Test Env)', 'models4/ablation_skipwk_ccs,accidents,airline,ssb,credit,baseball,basketball,movielens,seznam,financial.wt'}
{'models4/ablation_skipwk_tpch.wt', 'models4/ablation_skipwk_stack.wt', 'DBMS\n(Diff DB,Test Env)', 'DBMS\n(Test DB,Test Env)', 'models4/ablation_skipwk_ccs,accidents,airline,ssb,credit,baseball,basketball,movielens,seznam,financial.wt'}


In [20]:
#df = df[df["seed"] < 4]

In [21]:
single_df = df[~df["config"].str.contains("mt")]
single_df_imdb = single_df[single_df["config"].str.contains("noimdb")]
#tmp2 = df[df["config"].str.contains("mt")]
single_df_stack = single_df[single_df["config"].str.contains("nostack")]

In [22]:
single_df_imdb.groupby(["Estimator"])["Final-LatencyQError-test_unseen-mean"].describe().reset_index().sort_values(by="mean")

,Estimator,count,mean,std,min,25%,50%,75%,max
3,models4/ablation_skipwk_tpch.wt,4.0,6.892023,1.391858,5.438343,6.336824,6.670433,7.225633,8.788885
2,models4/ablation_skipwk_stack.wt,4.0,6.935965,3.204297,4.956653,5.152784,5.539335,7.322516,11.708536
1,"models4/ablation_skipwk_ccs,accidents,airline,...",4.0,9.043459,5.106741,5.283479,6.027694,7.195893,10.211658,16.498571
0,"DBMS\n(Test DB,Test Env)",1.0,17.142025,NaN,17.142025,17.142025,17.142025,17.142025,17.142025


In [23]:
single_df_stack.groupby(["Estimator"])["Final-LatencyQError-test_unseen-mean"].describe().reset_index().sort_values(by="mean")

,index,count,mean,std,min,25%,50%,75%,max
